# Classification

In this notebook, ...

In [475]:
import pandas as pd
import pickle
from tqdm import tqdm

In [289]:
from sklearn.svm import SVC
from sklearn import linear_model
from sklearn import set_config
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler, FunctionTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay, accuracy_score

In [290]:
set_config(display='diagram')

## Dataset

In [291]:
all_feat_OG = pd.read_csv('all_feat_final_new.csv')
all_feat_OG

,Unnamed: 0,full,clicked,n_token,n_sent,to_NoPhr_C,as_NoPhr_C,at_NoPhr_C,ra_NoVeP_C,ra_NoSuP_C,...,as_FTree_C,at_FTree_C,TokSenM_S,TokSenS_S,TokSenL_S,as_Token_C,as_Sylla_C,at_Sylla_C,as_Chara_C,at_Chara_C
0,0,Incoming Webhooks — Mattermost 5.11 documentat...,1,31,6,10.0,1.666667,0.322581,1.000000,10.0,...,5.833333,1.129032,186.0,13.638182,1.916545,5.166667,10.166667,1.967742,43.333333,8.387097
1,1,“Hello” Is Facebook's New Android-Only Social ...,0,28,4,14.0,3.500000,0.500000,2.333333,0.0,...,8.750000,1.250000,112.0,10.583005,2.403677,7.000000,10.750000,1.535714,47.250000,6.750000
2,2,Walkthrough | HTTP Event CollectorHEC tokens a...,0,41,5,15.0,3.000000,0.365854,1.666667,0.0,...,9.200000,1.121951,205.0,14.317821,2.307372,8.200000,13.600000,1.658537,62.400000,7.609756
3,3,College Scholarships - Scholarships.comMost co...,0,28,3,8.0,2.666667,0.285714,1.000000,0.0,...,11.333333,1.214286,84.0,9.165151,3.033103,9.333333,19.333333,2.071429,82.000000,8.785714
4,4,How to start sending email — Mailgun API docum...,0,31,5,9.0,1.800000,0.290323,0.642857,1.5,...,6.800000,1.096774,155.0,12.449900,2.133656,6.200000,10.200000,1.645161,50.800000,8.193548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,10287,"Science, Technology, Engineering, and Math, in...",1,27,5,17.0,3.400000,0.629630,5.666667,8.5,...,5.400000,1.000000,135.0,11.618950,2.047819,5.400000,11.200000,2.074074,37.400000,6.925926
10288,10288,Stem Kitchen SFSTEM Kitchen & Garden is a farm...,0,22,2,9.0,4.500000,0.409091,2.250000,9.0,...,12.500000,1.136364,44.0,6.633250,4.459432,11.000000,18.000000,1.636364,69.500000,6.318182
10289,10289,Stem Definition & Meaning - Merriam-WebsterKid...,0,22,3,12.0,4.000000,0.545455,1.714286,0.0,...,9.333333,1.272727,66.0,8.124038,2.813588,7.333333,13.000000,1.772727,56.333333,7.681818
10290,10290,Driven By StemStem Holdings has positioned its...,0,19,2,10.0,5.000000,0.526316,1.666667,0.0,...,12.500000,1.315789,38.0,6.164414,4.247928,9.500000,17.500000,1.842105,73.000000,7.684211


In [292]:
all_feat = all_feat_OG.copy()
all_feat.drop(columns = ['Unnamed: 0', 'full'], inplace = True, axis=1)
all_feat.rename(columns = {'clicked':'class'}, inplace = True)

In [293]:
all_feat

,class,n_token,n_sent,to_NoPhr_C,as_NoPhr_C,at_NoPhr_C,ra_NoVeP_C,ra_NoSuP_C,ra_NoPrP_C,ra_NoAjP_C,...,as_FTree_C,at_FTree_C,TokSenM_S,TokSenS_S,TokSenL_S,as_Token_C,as_Sylla_C,at_Sylla_C,as_Chara_C,at_Chara_C
0,1,31,6,10.0,1.666667,0.322581,1.000000,10.0,5.000000,5.0,...,5.833333,1.129032,186.0,13.638182,1.916545,5.166667,10.166667,1.967742,43.333333,8.387097
1,0,28,4,14.0,3.500000,0.500000,2.333333,0.0,3.500000,0.0,...,8.750000,1.250000,112.0,10.583005,2.403677,7.000000,10.750000,1.535714,47.250000,6.750000
2,0,41,5,15.0,3.000000,0.365854,1.666667,0.0,2.142857,0.0,...,9.200000,1.121951,205.0,14.317821,2.307372,8.200000,13.600000,1.658537,62.400000,7.609756
3,0,28,3,8.0,2.666667,0.285714,1.000000,0.0,4.000000,8.0,...,11.333333,1.214286,84.0,9.165151,3.033103,9.333333,19.333333,2.071429,82.000000,8.785714
4,0,31,5,9.0,1.800000,0.290323,0.642857,1.5,2.250000,4.5,...,6.800000,1.096774,155.0,12.449900,2.133656,6.200000,10.200000,1.645161,50.800000,8.193548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,1,27,5,17.0,3.400000,0.629630,5.666667,8.5,8.500000,0.0,...,5.400000,1.000000,135.0,11.618950,2.047819,5.400000,11.200000,2.074074,37.400000,6.925926
10288,0,22,2,9.0,4.500000,0.409091,2.250000,9.0,3.000000,9.0,...,12.500000,1.136364,44.0,6.633250,4.459432,11.000000,18.000000,1.636364,69.500000,6.318182
10289,0,22,3,12.0,4.000000,0.545455,1.714286,0.0,2.000000,0.0,...,9.333333,1.272727,66.0,8.124038,2.813588,7.333333,13.000000,1.772727,56.333333,7.681818
10290,0,19,2,10.0,5.000000,0.526316,1.666667,0.0,2.000000,0.0,...,12.500000,1.315789,38.0,6.164414,4.247928,9.500000,17.500000,1.842105,73.000000,7.684211


In [294]:
all_feat['class'].value_counts(normalize = True)

0    0.56986
1    0.43014
Name: class, dtype: float64

## Split train and test

In [295]:
X, y = all_feat.drop('class', axis=1), all_feat['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, stratify=y,
                                                   random_state=42)

In [296]:
X_train.shape, y_train.shape

((8233, 157), (8233,))

In [297]:
# Split the training set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)


In [298]:
X_train.shape, y_train.shape

((6586, 157), (6586,))

In [299]:
X_val.shape, y_val.shape

((1647, 157), (1647,))

## Models

### 1. Random Forest (rf)

In [368]:
%time
rfc_pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('model', RandomForestClassifier(random_state=42, n_jobs=-1))
])

rfc_pipeline.fit(X_train, y_train)

CPU times: user 8 µs, sys: 1 µs, total: 9 µs
Wall time: 21 µs


Pipeline(steps=[('scaler', StandardScaler()),
                ('model', RandomForestClassifier(n_jobs=-1, random_state=42))])

In [369]:
%time
pred_rfc = rfc_pipeline.predict(X_test)
cm_rf = confusion_matrix(y_true=y_test, y_pred=pred_rfc, labels= rfc_pipeline.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_rf, display_labels=rfc_pipeline.classes_)

CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 13.1 µs


In [302]:
FP=cm_rf[0,1]
FN=cm_rf[1,0]
TP=cm_rf[1,1]
TN=cm_rf[0,0]

TPR=TP/(TP+FN)
TNR=TN/(TN+FP)
FPR=FP/(FP+TN)
FNR=FN/(TP+FN)

In [303]:
TPR, TNR, FPR, FNR

(0.7618510158013544,
 0.8976982097186701,
 0.10230179028132992,
 0.2381489841986456)

In [304]:
print(classification_report(y_true=y_test, y_pred=pred_rfc,
                            labels= rfc_pipeline.classes_, digits=3))

              precision    recall  f1-score   support

           0      0.833     0.898     0.864      1173
           1      0.849     0.762     0.803       886

    accuracy                          0.839      2059
   macro avg      0.841     0.830     0.834      2059
weighted avg      0.840     0.839     0.838      2059



In [305]:
acc_rfc = accuracy_score(y_test, pred_rfc)
acc_rfc

0.8392423506556581

#### Tune RF Hyperparameter

The following code take a quite long to run

In [365]:
import time

In [427]:
# import time
%time
param_spec = {
    'model__n_estimators': [100, 200, 500],
    'model__criterion': ['gini', 'entropy'],
    'model__max_depth': [None, 20],
    'model__min_samples_split': [2], # -> 2,5 the same acc
    'model__max_features': ['sqrt', 'log2', None] #
}

rf_cv = GridSearchCV(estimator=rfc_pipeline, param_grid=param_spec,
                    n_jobs=-1, verbose=1, refit=True, cv=5)
# for i in tqdm(param_spec):
rf_cv.fit(X_train, y_train)

CPU times: user 4 µs, sys: 12 µs, total: 16 µs
Wall time: 29.1 µs
Fitting 5 folds for each of 36 candidates, totalling 180 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model',
                                        RandomForestClassifier(max_features=0.75,
                                                               n_jobs=-1,
                                                               random_state=42))]),
             n_jobs=-1,
             param_grid={'model__criterion': ['gini', 'entropy'],
                         'model__max_depth': [None, 20],
                         'model__max_features': ['sqrt', 'log2', None],
                         'model__min_samples_split': [2],
                         'model__n_estimators': [100, 200, 500]},
             verbose=1)

In [428]:
rf_cv.best_params_

{'model__criterion': 'entropy',
 'model__max_depth': None,
 'model__max_features': 'log2',
 'model__min_samples_split': 2,
 'model__n_estimators': 200}

In [429]:
y_pred_rfc = rf_cv.predict(X_val)
acc_val = accuracy_score(y_val, y_pred_rfc)
acc_val

0.843351548269581

In [437]:
yt_pred_rf_cv = rf_cv.predict(X_test)
print(classification_report(y_true=y_test, y_pred=yt_pred_rf_cv,
                            labels= svc_pipeline.classes_, digits=3))

              precision    recall  f1-score   support

           0      0.834     0.898     0.865      1173
           1      0.849     0.764     0.805       886

    accuracy                          0.840      2059
   macro avg      0.842     0.831     0.835      2059
weighted avg      0.841     0.840     0.839      2059



The code below is for testing different parameter so to know the one t0 use for grid search

In [425]:
%time
rfc_pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('model', RandomForestClassifier(random_state=42, n_jobs=-1, max_features= 0.75))
])

rfc_pipeline.fit(X_train, y_train)

CPU times: user 6 µs, sys: 1 µs, total: 7 µs
Wall time: 15 µs


Pipeline(steps=[('scaler', StandardScaler()),
                ('model',
                 RandomForestClassifier(max_features=0.75, n_jobs=-1,
                                        random_state=42))])

In [426]:
y_pred_rfc = rfc_pipeline.predict(X_val)
acc_val = accuracy_score(y_val, y_pred_rfc)
acc_val

0.8391013964784456

The commented cell bellow  are individual parameters with their corresponding values that I used to get the idea of the value range to be used for grid search

In [ ]:
# n_estimators
# 100 = 0.8445658773527626
# 500 = 0.8451730418943534
# 200 = 0.8439587128111719
# 100 = 0.8439587128111719 # default
# 50 = 0.833029751062538

In [ ]:
# criterion # ---> seem to not matter 
# entropy = 0.8415300546448088
# gini = 0.8439587128111719

In [ ]:
# max_depth
# 3 = 0.6533090467516697
# 7 = 0.7134183363691561
# 10 = 0.8233151183970856
# None = 0.8439587128111719 # default
# 20 = 0.8427443837279903
# 40 = 0.8439587128111719

In [ ]:
# min_samples_split 
# 2 = 0.8439587128111719 # default
# 5 = 0.8439587128111719

In [ ]:
# max_features
# auto = 0.8439587128111719
# sqrt = 0.8439587128111719
# log2 = 0.8415300546448088
# None = 0.8378870673952641
# 0.75 = 0.8391013964784456

## Feature importance

In [306]:
X_cols =  X_train.columns
X_scaled=StandardScaler().fit_transform(all_feat[X_cols])

In [432]:
# model=RandomForestClassifier(random_state=42, n_jobs=-1)
model=RandomForestClassifier(random_state=42, 
                             n_jobs=-1, 
                             n_estimators = 200, 
                             criterion = 'entropy',
                             max_depth = None,
                             min_samples_split = 2,
                             max_features = 'log2'
                             )
model.fit(X_scaled, all_feat['class'])

RandomForestClassifier(criterion='entropy', max_features='log2',
                       n_estimators=200, n_jobs=-1, random_state=42)

In [434]:
model.feature_importances_

array([0.01030672, 0.00240423, 0.00734123, 0.00837316, 0.00995613,
       0.00840326, 0.00480519, 0.00940728, 0.00341271, 0.00417132,
       0.00515021, 0.00604811, 0.0088486 , 0.00844456, 0.00402216,
       0.0073295 , 0.00277724, 0.0035531 , 0.0019978 , 0.00350712,
       0.00489439, 0.00454811, 0.00417595, 0.00372271, 0.00167842,
       0.00160536, 0.0045457 , 0.00659784, 0.00972214, 0.01018714,
       0.00739911, 0.00413713, 0.0028005 , 0.00279363, 0.00171025,
       0.00250567, 0.00333007, 0.00375066, 0.00337899, 0.00123025,
       0.00331084, 0.00126899, 0.00146608, 0.00259674, 0.00397275,
       0.00387351, 0.00335113, 0.00163998, 0.00309439, 0.0012861 ,
       0.00628206, 0.00915467, 0.01073346, 0.00737839, 0.00841097,
       0.00473011, 0.00195814, 0.00640103, 0.00401107, 0.00619007,
       0.00934418, 0.00706861, 0.0083553 , 0.00407494, 0.00156348,
       0.00460527, 0.00485556, 0.00859792, 0.01075004, 0.0082239 ,
       0.00834478, 0.00431493, 0.00164265, 0.00431199, 0.00202

In [435]:
FeatImp = pd.Series(model.feature_importances_, index=X_cols).sort_values(ascending=False)
FeatImp.head(40)

at_Chara_C    0.014162
to_FuncW_C    0.013975
at_Sylla_C    0.013774
at_FuncW_C    0.013334
FleschG_S     0.012751
to_AABiL_C    0.012473
ra_CoFuW_C    0.012365
at_TreeH_C    0.012325
at_AAKuL_C    0.011991
at_AAKuW_C    0.011947
as_Chara_C    0.011822
at_AABiL_C    0.011792
to_AAKuW_C    0.011208
AutoRea_S     0.011187
TokSenS_S     0.011184
at_ContW_C    0.011127
as_AABiL_C    0.011060
to_AAKuL_C    0.010917
as_AAKuW_C    0.010836
at_AACoL_C    0.010818
at_AjTag_C    0.010750
at_NoTag_C    0.010733
as_FuncW_C    0.010715
TokSenM_S     0.010668
at_AABrL_C    0.010666
at_FTree_C    0.010553
as_AACoL_C    0.010469
as_AAKuL_C    0.010385
n_token       0.010307
as_AABrL_C    0.010226
ra_PrNoP_C    0.010187
to_AACoL_C    0.010118
LinseaW_S     0.010097
UberTTR_S     0.010075
to_AABrL_C    0.009985
Gunning_S     0.009984
BiLoTTR_S     0.009957
at_NoPhr_C    0.009956
as_Sylla_C    0.009836
SimpTTR_S     0.009791
dtype: float64

In [436]:
FeatImp.tail(40)

as_AvTag_C    0.003309
ra_AvPrP_C    0.003094
to_CoTag_C    0.003064
ra_AvAjT_C    0.002972
ra_CoAvT_C    0.002810
ra_PrAjP_C    0.002801
ra_PrAvP_C    0.002794
ra_VeAjP_C    0.002777
as_AvPhr_C    0.002597
as_AjPhr_C    0.002506
n_sent        0.002404
SimpVeV_S     0.002299
ra_AvCoT_C    0.002284
CorrAvV_S     0.002134
to_AvTag_C    0.002023
to_SuPhr_C    0.001998
SquaAvV_S     0.001996
ra_NoSuT_C    0.001958
SimpAjV_S     0.001954
ra_SuNoT_C    0.001802
to_AjPhr_C    0.001710
ra_SuAjT_C    0.001679
ra_SuAjP_C    0.001678
ra_AjSuT_C    0.001643
ra_AvSuP_C    0.001640
ra_SuAvP_C    0.001605
at_SuTag_C    0.001582
ra_VeSuT_C    0.001563
to_AvPhr_C    0.001466
SimpAvV_S     0.001418
ra_SuVeT_C    0.001348
ra_AvAjP_C    0.001286
ra_AjAvP_C    0.001269
ra_AjSuP_C    0.001230
as_SuTag_C    0.001178
ra_CoSuT_C    0.001118
ra_SuCoT_C    0.000725
ra_AvSuT_C    0.000719
ra_SuAvT_C    0.000715
to_SuTag_C    0.000712
dtype: float64

## Feature groups

In [311]:
syntax = ['to_NoPhr_C',
'as_NoPhr_C',
'at_NoPhr_C',
'ra_NoVeP_C',
'ra_NoSuP_C',
'ra_NoPrP_C',
'ra_NoAjP_C',
'ra_NoAvP_C',
'to_VePhr_C',
'as_VePhr_C',
'at_VePhr_C',
'ra_VeNoP_C',
'ra_VeSuP_C',
'ra_VePrP_C',
'ra_VeAjP_C',
'ra_VeAvP_C',
'to_SuPhr_C',
'as_SuPhr_C',
'at_SuPhr_C',
'ra_SuNoP_C',
'ra_SuVeP_C',
'ra_SuPrP_C',
'ra_SuAjP_C',
'ra_SuAvP_C',
'to_PrPhr_C',
'as_PrPhr_C',
'at_PrPhr_C',
'ra_PrNoP_C',
'ra_PrVeP_C',
'ra_PrSuP_C',
'ra_PrAjP_C',
'ra_PrAvP_C',
'to_AjPhr_C',
'as_AjPhr_C',
'at_AjPhr_C',
'ra_AjNoP_C',
'ra_AjVeP_C',
'ra_AjSuP_C',
'ra_AjPrP_C',
'ra_AjAvP_C',
'to_AvPhr_C',
'as_AvPhr_C',
'at_AvPhr_C',
'ra_AvNoP_C',
'ra_AvVeP_C',
'ra_AvSuP_C',
'ra_AvPrP_C',
'ra_AvAjP_C',
'to_NoTag_C',
'as_NoTag_C',
'at_NoTag_C',
'ra_NoAjT_C',
'ra_NoVeT_C',
'ra_NoAvT_C',
'ra_NoSuT_C',
'ra_NoCoT_C',
'to_VeTag_C',
'as_VeTag_C',
'at_VeTag_C',
'ra_VeAjT_C',
'ra_VeNoT_C',
'ra_VeAvT_C',
'ra_VeSuT_C',
'ra_VeCoT_C',
'to_AjTag_C',
'as_AjTag_C',
'at_AjTag_C',
'ra_AjNoT_C',
'ra_AjVeT_C',
'ra_AjAvT_C',
'ra_AjSuT_C',
'ra_AjCoT_C',
'to_AvTag_C',
'as_AvTag_C',
'at_AvTag_C',
'ra_AvAjT_C',
'ra_AvNoT_C',
'ra_AvVeT_C',
'ra_AvSuT_C',
'ra_AvCoT_C',
'to_SuTag_C',
'as_SuTag_C',
'at_SuTag_C',
'ra_SuAjT_C',
'ra_SuNoT_C',
'ra_SuVeT_C',
'ra_SuAvT_C',
'ra_SuCoT_C',
'to_CoTag_C',
'as_CoTag_C',
'at_CoTag_C',
'ra_CoAjT_C',
'ra_CoNoT_C',
'ra_CoVeT_C',
'ra_CoAvT_C',
'ra_CoSuT_C',
'to_ContW_C',
'as_ContW_C',
'at_ContW_C',
'to_FuncW_C',
'as_FuncW_C',
'at_FuncW_C',
'ra_CoFuW_C',
'to_TreeH_C',
'as_TreeH_C',
'at_TreeH_C',
'to_FTree_C',
'as_FTree_C',
'at_FTree_C']

In [312]:
len(syntax), len(syntax) == len(set(syntax))

(109, True)

In [313]:
lex_sem = ['SimpNoV_S',
'SquaNoV_S',
'CorrNoV_S',
'SimpVeV_S',
'SquaVeV_S',
'CorrVeV_S',
'SimpAjV_S',
'SquaAjV_S',
'CorrAjV_S',
'SimpAvV_S',
'SquaAvV_S',
'CorrAvV_S',
'SimpTTR_S',
'CorrTTR_S',
'BiLoTTR_S',
'UberTTR_S',
'MTLDTTR_S',
'to_AAKuW_C',
'as_AAKuW_C',
'at_AAKuW_C',
'to_AAKuL_C',
'as_AAKuL_C',
'at_AAKuL_C',
'to_AABiL_C',
'as_AABiL_C',
'at_AABiL_C',
'to_AABrL_C',
'as_AABrL_C',
'at_AABrL_C',
'to_AACoL_C',
'as_AACoL_C',
'at_AACoL_C']

In [314]:
len(lex_sem), len(lex_sem) == len(set(lex_sem))

(32, True)

In [315]:
shallow_trad = ['TokSenM_S',
'TokSenS_S',
'TokSenL_S',
'as_Token_C',
'as_Sylla_C',
'at_Sylla_C',
'as_Chara_C',
'at_Chara_C',     
'SmogInd_S',
'ColeLia_S',
'Gunning_S',
'AutoRea_S',
'FleschG_S',
'LinseaW_S']

In [346]:
preprocess = ['n_token',
              'n_sent']

In [440]:
AllF = list(X_train.columns)
len(AllF)

157

In [316]:
len(shallow_trad), len(shallow_trad) == len(set(shallow_trad))

(14, True)

In [317]:
# check a variable exist in the data
'to_AAKuW_C' in all_feat.columns

True

In [318]:
all_feat_sets = shallow_trad + lex_sem + syntax
len(all_feat_sets), len(all_feat_sets) == len(set(all_feat_sets))

(155, True)

In [319]:
# all_feat_sets : total feat = 157, we 155 because of prep feat = n_token and n_sent

## Function

In [481]:
def train_and_evaluate_model(cols_of_interest, name):
    
    model_pipe = Pipeline(steps=[
        ('select_variable', FunctionTransformer(lambda df: df[cols_of_interest])),
        ('scale', StandardScaler()),
        ('model', RandomForestClassifier(random_state=42, 
                                         n_jobs=-1,  
                                         n_estimators = 200, 
                                         criterion = 'entropy',
                                         max_depth = None,
                                         min_samples_split = 2,
                                         max_features = 'log2'))
        
    ])
    
    model_pipe.fit(X_train, y_train)
    
    #best_model=model_pipe.named_steps['crossvalidate'].best_params_
    
    n_feat=model_pipe.named_steps['model'].n_features_in_
    print(f'Number of Feature for {name}: {n_feat}')
    
    #make pred
    y_test_pred=model_pipe.predict(X_test)
    
    # save the predicted values to be used for statistical significance
    pickle.dump( y_test_pred, open( "pred_results/pred_" + str(name) + ".p", "wb" ))
    
    cm=confusion_matrix(y_true=y_test, y_pred=y_test_pred, labels= model_pipe.classes_)
    acc = accuracy_score(y_true=y_test, y_pred=y_test_pred)
    
    FP=cm[0,1]
    FN=cm[1,0]
    TP=cm[1,1]
    TN=cm[0,0]

    TPR=TP/(TP+FN)
    TNR=TN/(TN+FP)
    FPR=FP/(FP+TN)
    FNR=FN/(TP+FN)
    
    columns=['Type', 'Accuracy', 'TPR', 'TPR', 'FPR', 'FNR']
    entries=[[name, acc, TPR, TNR, FPR, FNR]]
    scores = pd.DataFrame(data=entries, columns=columns)
    
    return scores

### Individual feature set

In [482]:
all_feat = train_and_evaluate_model(AllF, 'all_feat')
shallow_trad_res = train_and_evaluate_model(shallow_trad, "shallow_trad")
lex_sem_res = train_and_evaluate_model(lex_sem, "lex_sem")
syntax_res = train_and_evaluate_model(syntax, "syntax")

Number of Feature for all_feat: 157
Number of Feature for shallow_trad: 14
Number of Feature for lex_sem: 32
Number of Feature for syntax: 109


In [483]:
results_1 = pd.concat([all_feat, shallow_trad_res, lex_sem_res, syntax_res], ignore_index=True)
results_1

,Type,Accuracy,TPR,TPR,FPR,FNR
0,all_feat,0.840214,0.764108,0.897698,0.102302,0.235892
1,shallow_trad,0.813987,0.767494,0.849105,0.150895,0.232506
2,lex_sem,0.839242,0.761851,0.897698,0.102302,0.238149
3,syntax,0.840699,0.765237,0.897698,0.102302,0.234763


### Combine

In [484]:
shallow_trad_AND_lex_sem = shallow_trad + lex_sem
shallow_trad_AND_syntax = shallow_trad + syntax
shallow_trad_AND_lex_sem_AND_syntax = shallow_trad + lex_sem + syntax

In [485]:
all_feat = train_and_evaluate_model(AllF, 'all_feat')
shallow_trad_lex_sem_syntax = train_and_evaluate_model(shallow_trad_AND_lex_sem_AND_syntax, "shallow_trad_lex_sem_syntax")
shallow_trad_AND_lex_sem = train_and_evaluate_model(shallow_trad_AND_lex_sem, "shallow_trad_AND_lex_sem")
shallow_trad_AND_syntax = train_and_evaluate_model(shallow_trad_AND_syntax, "shallow_trad_AND_syntax")


Number of Feature for all_feat: 157
Number of Feature for shallow_trad_lex_sem_syntax: 155
Number of Feature for shallow_trad_AND_lex_sem: 46
Number of Feature for shallow_trad_AND_syntax: 123


In [486]:
results_2 = pd.concat([all_feat, shallow_trad_AND_lex_sem, shallow_trad_AND_syntax, shallow_trad_lex_sem_syntax], ignore_index=True)
results_2

,Type,Accuracy,TPR,TPR,FPR,FNR
0,all_feat,0.840214,0.764108,0.897698,0.102302,0.235892
1,shallow_trad_AND_lex_sem,0.836814,0.762980,0.892583,0.107417,0.237020
2,shallow_trad_AND_syntax,0.841671,0.764108,0.900256,0.099744,0.235892
3,shallow_trad_lex_sem_syntax,0.840699,0.764108,0.898551,0.101449,0.235892


### Remove 

In [487]:
all_cols = list(X.columns)
len(all_cols)

157

In [488]:
X.size

1615844

In [489]:
# remove syntax
all_feat_minus_syntax = [ele for ele in all_cols if ele not in syntax]
# remove lex_sem 
all_feat_minus_lex_sem = [ele for ele in all_cols if ele not in lex_sem]
# remove shallow_trad
all_feat_minus_shallow_trad = [ele for ele in all_cols if ele not in shallow_trad]
# remove preprocess
all_feat_minus_preprocess = [ele for ele in all_cols if ele not in preprocess]

In [490]:
len(all_feat_minus_syntax), len(all_feat_minus_lex_sem), len(all_feat_minus_shallow_trad), len(all_feat_minus_preprocess)


(48, 125, 143, 155)

In [493]:
all_feat = train_and_evaluate_model(AllF, 'all_feat') 
all_feat_minus_syntax_res = train_and_evaluate_model(all_feat_minus_syntax, 'all_feat_minus_syntax')
all_feat_minus_lex_sem_res = train_and_evaluate_model(all_feat_minus_lex_sem, 'all_feat_minus_lex_sem')
all_feat_minus_shallow_trad_res = train_and_evaluate_model(all_feat_minus_shallow_trad, 'all_feat_minus_shallow_trad')
all_feat_minus_preprocess_res = train_and_evaluate_model(all_feat_minus_preprocess, 'all_feat_minus_preprocess')


Number of Feature for all_feat: 157
Number of Feature for all_feat_minus_syntax: 48
Number of Feature for all_feat_minus_lex_sem: 125
Number of Feature for all_feat_minus_shallow_trad: 143
Number of Feature for all_feat_minus_preprocess: 155


In [494]:
Results_3 = pd.concat([all_feat,
                       all_feat_minus_syntax_res, 
                       all_feat_minus_lex_sem_res, 
                       all_feat_minus_shallow_trad_res,
                       all_feat_minus_preprocess_res], 
                      ignore_index = True)
Results_3

,Type,Accuracy,TPR,TPR,FPR,FNR
0,all_feat,0.840214,0.764108,0.897698,0.102302,0.235892
1,all_feat_minus_syntax,0.838757,0.765237,0.894288,0.105712,0.234763
2,all_feat_minus_lex_sem,0.836814,0.759594,0.895141,0.104859,0.240406
3,all_feat_minus_shallow_trad,0.842156,0.764108,0.901108,0.098892,0.235892
4,all_feat_minus_preprocess,0.841185,0.762980,0.900256,0.099744,0.237020


### 2. Support Vector Machine (SVM)


In [333]:

svc_pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('model', SVC())
])

svc_pipeline.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('model', SVC())])

In [334]:
pred_svc = svc_pipeline.predict(X_test)
cm_svm = confusion_matrix(y_true=y_test, y_pred=pred_svc, labels= svc_pipeline.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_svm, display_labels=svc_pipeline.classes_)


In [335]:
FP=cm_svm[0,1]
FN=cm_svm[1,0]
TP=cm_svm[1,1]
TN=cm_svm[0,0]

TPR=TP/(TP+FN)
TNR=TN/(TN+FP)
FPR=FP/(FP+TN)
FNR=FN/(TP+FN)

In [336]:
TPR, TNR, FPR, FNR

(0.582392776523702, 0.907075873827792, 0.09292412617220801, 0.417607223476298)

In [337]:
print(classification_report(y_true=y_test, y_pred=pred_svc,
                            labels= svc_pipeline.classes_, digits=3))

              precision    recall  f1-score   support

           0      0.742     0.907     0.816      1173
           1      0.826     0.582     0.683       886

    accuracy                          0.767      2059
   macro avg      0.784     0.745     0.750      2059
weighted avg      0.778     0.767     0.759      2059



#### svm tune hyperparameter

In [495]:
linear_kernel = {
    'model__C': [0.1, 0.5, 1, 5, 10], 
    'model__kernel': ['linear']
}

rbf_kernel ={
    'model__C': [0.1, 0.5, 1, 5, 10], 
    'model__kernel': ['rbf'],
    'model__gamma': ['scale', 'auto', 0.001, 0.00001]
}

poly_kernel ={
    'model__C': [0.1, 0.5, 1, 5, 10], 
    'model__kernel': ['poly'],
    'model__degree': [2,3],
    'model__gamma': ['scale', 'auto', 0.001, 0.00001]
}


param = [
    linear_kernel,
    rbf_kernel,
    poly_kernel
]

svc_cv = GridSearchCV(estimator=svc_pipeline, param_grid=param, n_jobs=-1, verbose=2)

svc_cv.fit(X_train, y_train)

Fitting 5 folds for each of 65 candidates, totalling 325 fits


GridSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model', SVC())]),
             n_jobs=-1,
             param_grid=[{'model__C': [0.1, 0.5, 1, 5, 10],
                          'model__kernel': ['linear']},
                         {'model__C': [0.1, 0.5, 1, 5, 10],
                          'model__gamma': ['scale', 'auto', 0.001, 1e-05],
                          'model__kernel': ['rbf']},
                         {'model__C': [0.1, 0.5, 1, 5, 10],
                          'model__degree': [2, 3],
                          'model__gamma': ['scale', 'auto', 0.001, 1e-05],
                          'model__kernel': ['poly']}],
             verbose=2)

In [501]:
svc_cv.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()), ('model', SVC(C=5))])

In [502]:
svc_cv.best_params_

{'model__C': 5, 'model__gamma': 'scale', 'model__kernel': 'rbf'}

In [503]:
new_ys_pred = svc_cv.best_estimator_.predict(X_test)

In [504]:
print(classification_report(y_true=y_test, y_pred=new_ys_pred, labels= svc_pipeline.classes_))

              precision    recall  f1-score   support

           0       0.80      0.86      0.83      1173
           1       0.79      0.72      0.76       886

    accuracy                           0.80      2059
   macro avg       0.80      0.79      0.79      2059
weighted avg       0.80      0.80      0.80      2059



### 3. MLP Classifier

In [374]:
%%time
mlp_pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('model', MLPClassifier(random_state=42, max_iter=300, validation_fraction = 0.1, verbose = True))
])
mlp_pipeline.fit(X_train, y_train)

Iteration 1, loss = 0.69060082
Iteration 2, loss = 0.60727773
Iteration 3, loss = 0.57793559
Iteration 4, loss = 0.55789001
Iteration 5, loss = 0.53950040
Iteration 6, loss = 0.52580841
Iteration 7, loss = 0.51664902
Iteration 8, loss = 0.50400588
Iteration 9, loss = 0.49195350
Iteration 10, loss = 0.48388476
Iteration 11, loss = 0.47656125
Iteration 12, loss = 0.47220858
Iteration 13, loss = 0.46199542
Iteration 14, loss = 0.45434885
Iteration 15, loss = 0.44759090
Iteration 16, loss = 0.43945162
Iteration 17, loss = 0.43622879
Iteration 18, loss = 0.42860044
Iteration 19, loss = 0.42432324
Iteration 20, loss = 0.41973531
Iteration 21, loss = 0.41165876
Iteration 22, loss = 0.40912925
Iteration 23, loss = 0.40264583
Iteration 24, loss = 0.39879836
Iteration 25, loss = 0.39572889
Iteration 26, loss = 0.39254649
Iteration 27, loss = 0.38553623
Iteration 28, loss = 0.38394359
Iteration 29, loss = 0.38130523
Iteration 30, loss = 0.37158058
Iteration 31, loss = 0.37206072
Iteration 32, los

Pipeline(steps=[('scaler', StandardScaler()),
                ('model',
                 MLPClassifier(max_iter=300, random_state=42, verbose=True))])

In [360]:
pred_mlp = mlp_pipeline.predict(X_test)
cm_mlp = confusion_matrix(y_true=y_test, y_pred=pred_mlp, labels = mlp_pipeline.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_mlp, display_labels=mlp_pipeline.classes_)

In [361]:
FP=cm_mlp[0,1]
FN=cm_mlp[1,0]
TP=cm_mlp[1,1]
TN=cm_mlp[0,0]

TPR=TP/(TP+FN)
TNR=TN/(TN+FP)
FPR=FP/(FP+TN)
FNR=FN/(TP+FN)

In [362]:
TPR, TNR, FPR, FNR

(0.7720090293453724,
 0.7945439045183291,
 0.20545609548167093,
 0.22799097065462753)

In [363]:
print(classification_report(y_true=y_test, y_pred=pred_mlp,
                            labels= mlp_pipeline.classes_, digits=3))
   

              precision    recall  f1-score   support

           0      0.822     0.795     0.808      1173
           1      0.739     0.772     0.755       886

    accuracy                          0.785      2059
   macro avg      0.781     0.783     0.782      2059
weighted avg      0.786     0.785     0.785      2059



#### Tune MLP HP

In [ ]:
# hidden_layer_sizes = [(10,), (20,), (30,), (40,)] # default=(100,)
# activation = {‘identity’, ‘logistic’, ‘tanh’, ‘relu’} #  default=’relu’
# solver = {‘lbfgs’, ‘sgd’, ‘adam’} # default=’adam’
# alphafloat =  default=0.0001
# batch_sizeint = [batch_size=min(200, n_samples)]# default=’auto’
# learning_rate = [‘constant’, ‘invscaling’, ‘adaptive’], default=’constant’


In [353]:
param_grid = {
    'hidden_layer_sizes': [(30,), (50,), (100,), (100,)],
    'activation': ['logistic', 'relu', 'relu'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'adaptive', 'invscaling']
}

mlp = MLPClassifier(max_iter=1000)

# Create a GridSearchCV object to perform hyperparameter tuning
grid_search = GridSearchCV(mlp, param_grid=param_grid, cv=5, verbose=1)

# Train the GridSearchCV object on the validating data
grid_search.fit(X_val, y_val)

# Print the best hyperparameters found by GridSearchCV
print("Best hyperparameters:", grid_search.best_params_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best hyperparameters: {'activation': 'logistic', 'alpha': 0.001, 'hidden_layer_sizes': (100,), 'learning_rate': 'invscaling'}


In [355]:
best_mlp = grid_search.best_estimator_
y_pred = best_mlp.predict(X_test)
accuracy = best_mlp.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.7095677513355998
